In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys
module_path = os.path.abspath(os.path.join('../../../src/'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
from datasets.datasethandler import DatasetHandler
datasetHandler = DatasetHandler()

In [3]:
import torch
import config
import pytorch_lightning as pl
from classification.models.SpectrogramCNN import SpectrogramCNNPLModule
from classification.models.DeepRecursiveCNN import DeepRecursiveCNNPLModule

In [ ]:
hparams = {
    "batch_size": 2,
    "learning_rate": 0.001,
    "weight_decay": 0.01,
    "lr_decay": 1
}

model = DeepRecursiveCNNPLModule(hparams)
model.prepare_data()
datasetHandler.load(model, 'training')
datasetHandler.load(model, 'validation')

trainer = pl.Trainer(
    max_epochs=30,
    #logger= loggers.TensorBoardLogger(config.LOG_DIR, name=type(model)._name_),
    gpus=1 if torch.cuda.is_available() else None
)

trainer.fit(model)

Load: /nfs/students/summer-term-2020/project-4/data/dataset1/dataset_resampled/training.p
Load: /nfs/students/summer-term-2020/project-4/data/dataset1/dataset_resampled/validation.p


GPU available: True, used: True
No environment variable for node rank defined. Set as 0.
CUDA_VISIBLE_DEVICES: [0]
/nfs/students/summer-term-2020/project-4/yan/.conda/envs/cnn_project/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:23: RuntimeWarning: You have defined a `val_dataloader()` and have defined a `validation_step()`, you may also want to define `validation_epoch_end()` for accumulating stats.
  warnings.warn(*args, **kwargs)
Set SLURM handle signals.

   | Name        | Type             | Params
---------------------------------------------
0  | model       | DeepRecursiveCNN | 2 M   
1  | model.bn0   | BatchNorm1d      | 2     
2  | model.conv1 | Conv1d           | 10 K  
3  | model.bn1   | BatchNorm1d      | 256   
4  | model.pool1 | MaxPool1d        | 0     
5  | model.drop1 | Dropout          | 0     
6  | model.conv2 | Conv1d           | 98 K  
7  | model.bn2   | BatchNorm1d      | 512   
8  | model.pool2 | MaxPool1d        | 0     
9  | model.con

Val-Acc=0.0011855364552459987


/nfs/students/summer-term-2020/project-4/yan/.conda/envs/cnn_project/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:23: UserWarning: The dataloader, train dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


Val-Acc=0.6330764671013633


Val-Acc=0.7557794902193242


Val-Acc=0.7889745109662122


Val-Acc=0.8180201541197392


Val-Acc=0.7854179016004742


Val-Acc=0.6751630112625964


In [5]:
# model.save("/nfs/students/summer-term-2020/project-4/SAVED_MODELS/SpectrogramCNN/vanilla.p")